Data Set Information:

We perform energy analysis using 12 different building shapes simulated in Ecotect. The buildings differ with respect to the glazing area, the glazing area distribution, and the orientation, amongst other parameters. We simulate various settings as functions of the afore-mentioned characteristics to obtain 768 building shapes. The dataset comprises 768 samples and 8 features, aiming to predict two real valued responses. It can also be used as a multi-class classification problem if the response is rounded to the nearest integer.


Attribute Information:

The dataset contains eight attributes (or features, denoted by X1...X8) and two responses (or outcomes, denoted by y1 and y2). The aim is to use the eight features to predict each of the two responses. 

Specifically: 
- X1 Relative Compactness 
- X2 Surface Area 
- X3 Wall Area 
- X4 Roof Area 
- X5 Overall Height 
- X6 Orientation 
- X7 Glazing Area 
- X8 Glazing Area Distribution 
- y1 Heating Load 
- y2 Cooling Load

In [1]:
import numpy as np 
import pandas as pd
#import dataset
df = pd.read_csv("/Users/philippebeliveau/Desktop/Notebook_Jupyter_&_R/Dataset/UCL_Dataset.csv")

df = df.drop(['X8','Unnamed: 10', 'Unnamed: 11'], axis=1)
df.head()

,X1,X2,X3,X4,X5,X6,X7,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2.0,0.0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3.0,0.0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4.0,0.0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5.0,0.0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2.0,0.0,20.84,28.28


Here we see that we have two ouputs Y1 and Y2, which is a good candidate for a multiple output neural network

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

## EDA

My job here is to explore the releationship between the features itself, between the features and target and identify specific features caracteristic that could be leverage to better 'cluster/predict' the target variable. In the case that I find specific releationship between the features and target variable, maybe I could attempt to use a residual connection so that specific inputs pass throught the network without having to pass throught the network 

In [9]:
df.isna().sum() # We see that there is 528 nan value per variables, so I will just omit them
df = df.dropna()
df.isna().sum().sum()

0

In [ ]:
# We see that the variables are on different scale 
# We also see that X2, X3, X4 seems to contain lots of information as of their high variance
print(describe)
# Another important point is that there is lots of resemblance between the value of Y1
# and Y2 

describe = df.describe().iloc[:, 7:]
difference = describe.iloc[:,0] - describe.iloc[:,1]
print(f'We see that the is not much difference between the two outputs {difference}')

In [ ]:
scatter_matrix = pd.plotting.scatter_matrix(df, figsize=[8,8], s=5, marker="D")

# Looking at the releationship between features and target, we see that the features are 
# having the same releationship between the two outputs, this could have been predicted looking
# at the difference in value

In [ ]:
# Looking at the distribution of the target variable... 
for i in df[['Y1', 'Y2']]: 
    plt.hist(df[i], density=True, facecolor='g', alpha=0.75)
    plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
    plt.xlim(0, 50)
    plt.ylim(0, 0.03)
    plt.grid(True)
    plt.show()

In [ ]:
for i in df.iloc[:8]: 
    plt.figure()
    plt.hist(df[i])
    plt.show()

# Prepare the dataset

In [3]:
from sklearn.model_selection import train_test_split

In [10]:
def format_output(data):
    y1 = data.pop('Y1')
    y1 = np.array(y1)
    y2 = data.pop('Y2')
    y2 = np.array(y2)
    return y1, y2

train, test = train_test_split(df, test_size=0.2)

train_Y = format_output(train)
test_Y = format_output(test)

In [11]:
# standarize of the X 
def standardize(x):
    x_norm = (x - np.mean(x)) /np.std(x)
    return x_norm

In [12]:
norm_train = standardize(train)
norm_test = standardize(test)

/Users/philippebeliveau/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3370: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [14]:
norm_train

,X1,X2,X3,X4,X5,X6,X7
585,0.880697,-0.929491,-0.562025,-0.657213,1.003263,-0.444882,1.233585
582,1.251517,-1.202295,0.010253,-1.199226,1.003263,0.450717,1.233585
673,1.993156,-1.747905,-0.562025,-1.470232,1.003263,-0.444882,1.233585
502,-0.046352,-0.111077,2.299361,-1.199226,1.003263,0.450717,0.114794
268,-0.509877,0.434532,-1.134302,0.968826,-0.996748,-1.340481,-1.003996
...,...,...,...,...,...,...,...
43,-1.158812,1.252946,0.582530,0.968826,-0.996748,1.346315,-1.749857
171,-0.231762,0.161728,-1.706579,0.968826,-0.996748,1.346315,-1.003996
330,-1.158812,1.252946,0.582530,0.968826,-0.996748,0.450717,0.114794
282,-1.158812,1.252946,0.582530,0.968826,-0.996748,0.450717,-1.003996


# Tensorflow

## Sequential API

In [15]:
# Define the input 
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# Let's start by doing the "Hello world" of Deep learnin with the sequential API
def model(x, y, x_t, y_t): 
    early_stop = EarlyStopping(monitor='val_loss', patience=5)
    
    model = Sequential()
    model.add(Dense(20, input_shape=(7,), activation='relu'))

    model.add(Dense(1, activation = 'relu'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    model.fit(x, y, validation_data=(x_t, y_t), epochs=20, 
                        callbacks=[early_stop])
    preds = model.predict(x)
    
    return model.summary()

In [ ]:
model(X_train, y1_train, X_test, y1_test)

## Functional API

In [ ]:
# Inception model where the layer 1 is connected to 4 layers at the same time, which then 
# those 4 layers are concatenated 

layer_1 = Dense(32)
layer2_1 = Dense(32)(layer_1)
layer2_2 = Dense(32)(layer_1)
layer2_3 = Dense(32)(layer_1)
layer2_4 = Dense(32)(layer_1)
merge = Concatenate([layer2_1, layer2_2, layer2_3, layer2_4])

In [16]:
# Using the functional API and having a multi class output
input_layer = Input(shape=(len(norm_train.columns),))
dense_1 = Dense(32, activation='relu')(input_layer)
y1_output = Dense(1, activation = 'relu', name='y1_output')(dense_1)
dense_2 = Dense(16, activation = 'relu')(dense_1)
y2_output = Dense(1, activation= 'relu', name='y2_output')(dense_2)

model = Model(inputs=input_layer, outputs=[y1_output, y2_output])
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 7)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           256         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 16)           528         dense[0][0]                      
__________________________________________________________________________________________________
y1_output (Dense)               (None, 1)            33          dense[0][0]                      
______________________________________________________________________________________________

In [17]:
model.compile(optimizer='rmsprop', metrics={'y1_output':tf.keras.metrics.RootMeanSquaredError(),
                                            'y2_output': tf.keras.metrics.RootMeanSquaredError()},

              loss = {'y1_output':'mse', 'y2_output':'mse'})

In [18]:
history = model.fit(norm_train, train_Y,
                    epochs=500, batch_size=10, validation_data=(norm_test, test_Y))

Epoch 1/500
62/62 [==============================] - 1s 3ms/step - loss: 1211.9958 - y1_output_loss: 550.6337 - y2_output_loss: 661.3623 - y1_output_root_mean_squared_error: 23.4656 - y2_output_root_mean_squared_error: 25.7170 - val_loss: 1220.4030 - val_y1_output_loss: 561.7368 - val_y2_output_loss: 658.6664 - val_y1_output_root_mean_squared_error: 23.7010 - val_y2_output_root_mean_squared_error: 25.6645
Epoch 2/500
62/62 [==============================] - 0s 1ms/step - loss: 1065.7703 - y1_output_loss: 496.3748 - y2_output_loss: 569.3954 - y1_output_root_mean_squared_error: 22.2795 - y2_output_root_mean_squared_error: 23.8620 - val_loss: 1042.8392 - val_y1_output_loss: 501.8675 - val_y2_output_loss: 540.9717 - val_y1_output_root_mean_squared_error: 22.4024 - val_y2_output_root_mean_squared_error: 23.2588
Epoch 3/500
62/62 [==============================] - 0s 1ms/step - loss: 866.8930 - y1_output_loss: 435.5861 - y2_output_loss: 431.3071 - y1_output_root_mean_squared_error: 20.8707 -

62/62 [==============================] - 0s 1ms/step - loss: 25.0532 - y1_output_loss: 12.0907 - y2_output_loss: 12.9625 - y1_output_root_mean_squared_error: 3.4772 - y2_output_root_mean_squared_error: 3.6003 - val_loss: 25.1854 - val_y1_output_loss: 12.4549 - val_y2_output_loss: 12.7305 - val_y1_output_root_mean_squared_error: 3.5292 - val_y2_output_root_mean_squared_error: 3.5680
Epoch 22/500
62/62 [==============================] - 0s 1ms/step - loss: 23.9409 - y1_output_loss: 11.5953 - y2_output_loss: 12.3457 - y1_output_root_mean_squared_error: 3.4052 - y2_output_root_mean_squared_error: 3.5136 - val_loss: 24.8715 - val_y1_output_loss: 12.2060 - val_y2_output_loss: 12.6655 - val_y1_output_root_mean_squared_error: 3.4937 - val_y2_output_root_mean_squared_error: 3.5589
Epoch 23/500
62/62 [==============================] - 0s 977us/step - loss: 22.9561 - y1_output_loss: 11.1364 - y2_output_loss: 11.8196 - y1_output_root_mean_squared_error: 3.3371 - y2_output_root_mean_squared_error: 

Epoch 42/500
62/62 [==============================] - 0s 1ms/step - loss: 17.5162 - y1_output_loss: 8.4336 - y2_output_loss: 9.0826 - y1_output_root_mean_squared_error: 2.9041 - y2_output_root_mean_squared_error: 3.0137 - val_loss: 18.7086 - val_y1_output_loss: 9.0509 - val_y2_output_loss: 9.6577 - val_y1_output_root_mean_squared_error: 3.0085 - val_y2_output_root_mean_squared_error: 3.1077
Epoch 43/500
62/62 [==============================] - 0s 1ms/step - loss: 17.5401 - y1_output_loss: 8.4612 - y2_output_loss: 9.0789 - y1_output_root_mean_squared_error: 2.9088 - y2_output_root_mean_squared_error: 3.0131 - val_loss: 17.8032 - val_y1_output_loss: 8.6367 - val_y2_output_loss: 9.1664 - val_y1_output_root_mean_squared_error: 2.9388 - val_y2_output_root_mean_squared_error: 3.0276
Epoch 44/500
62/62 [==============================] - 0s 966us/step - loss: 17.3837 - y1_output_loss: 8.4262 - y2_output_loss: 8.9575 - y1_output_root_mean_squared_error: 2.9028 - y2_output_root_mean_squared_erro

Epoch 63/500
62/62 [==============================] - 0s 1ms/step - loss: 16.2921 - y1_output_loss: 8.1735 - y2_output_loss: 8.1187 - y1_output_root_mean_squared_error: 2.8589 - y2_output_root_mean_squared_error: 2.8493 - val_loss: 16.9656 - val_y1_output_loss: 8.4794 - val_y2_output_loss: 8.4862 - val_y1_output_root_mean_squared_error: 2.9119 - val_y2_output_root_mean_squared_error: 2.9131
Epoch 64/500
62/62 [==============================] - 0s 1ms/step - loss: 16.2241 - y1_output_loss: 8.1772 - y2_output_loss: 8.0470 - y1_output_root_mean_squared_error: 2.8596 - y2_output_root_mean_squared_error: 2.8367 - val_loss: 17.9852 - val_y1_output_loss: 8.5978 - val_y2_output_loss: 9.3874 - val_y1_output_root_mean_squared_error: 2.9322 - val_y2_output_root_mean_squared_error: 3.0639
Epoch 65/500
62/62 [==============================] - 0s 981us/step - loss: 16.2000 - y1_output_loss: 8.1395 - y2_output_loss: 8.0605 - y1_output_root_mean_squared_error: 2.8530 - y2_output_root_mean_squared_erro

Epoch 84/500
62/62 [==============================] - 0s 1ms/step - loss: 15.2107 - y1_output_loss: 8.0250 - y2_output_loss: 7.1857 - y1_output_root_mean_squared_error: 2.8328 - y2_output_root_mean_squared_error: 2.6806 - val_loss: 17.5321 - val_y1_output_loss: 8.5219 - val_y2_output_loss: 9.0102 - val_y1_output_root_mean_squared_error: 2.9192 - val_y2_output_root_mean_squared_error: 3.0017
Epoch 85/500
62/62 [==============================] - 0s 991us/step - loss: 15.3104 - y1_output_loss: 8.0647 - y2_output_loss: 7.2457 - y1_output_root_mean_squared_error: 2.8398 - y2_output_root_mean_squared_error: 2.6918 - val_loss: 16.2938 - val_y1_output_loss: 8.2482 - val_y2_output_loss: 8.0456 - val_y1_output_root_mean_squared_error: 2.8720 - val_y2_output_root_mean_squared_error: 2.8365
Epoch 86/500
62/62 [==============================] - 0s 1ms/step - loss: 15.0726 - y1_output_loss: 8.0290 - y2_output_loss: 7.0435 - y1_output_root_mean_squared_error: 2.8336 - y2_output_root_mean_squared_erro

...

Epoch 399/500
62/62 [==============================] - 0s 1ms/step - loss: 9.2727 - y1_output_loss: 6.4781 - y2_output_loss: 2.7946 - y1_output_root_mean_squared_error: 2.5452 - y2_output_root_mean_squared_error: 1.6717 - val_loss: 10.3619 - val_y1_output_loss: 6.7182 - val_y2_output_loss: 3.6437 - val_y1_output_root_mean_squared_error: 2.5919 - val_y2_output_root_mean_squared_error: 1.9089
Epoch 400/500
62/62 [==============================] - 0s 1ms/step - loss: 9.1880 - y1_output_loss: 6.4589 - y2_output_loss: 2.7291 - y1_output_root_mean_squared_error: 2.5414 - y2_output_root_mean_squared_error: 1.6520 - val_loss: 10.7674 - val_y1_output_loss: 6.6961 - val_y2_output_loss: 4.0713 - val_y1_output_root_mean_squared_error: 2.5877 - val_y2_output_root_mean_squared_error: 2.0177
Epoch 401/500
62/62 [==============================] - 0s 1ms/step - loss: 9.0396 - y1_output_loss: 6.3619 - y2_output_loss: 2.6777 - y1_output_root_mean_squared_error: 2.5223 - y2_output_root_mean_squared_error:

Epoch 420/500
62/62 [==============================] - 0s 1ms/step - loss: 8.8980 - y1_output_loss: 6.1108 - y2_output_loss: 2.7873 - y1_output_root_mean_squared_error: 2.4720 - y2_output_root_mean_squared_error: 1.6695 - val_loss: 9.8654 - val_y1_output_loss: 6.1770 - val_y2_output_loss: 3.6884 - val_y1_output_root_mean_squared_error: 2.4854 - val_y2_output_root_mean_squared_error: 1.9205
Epoch 421/500
62/62 [==============================] - 0s 1ms/step - loss: 8.8383 - y1_output_loss: 6.1190 - y2_output_loss: 2.7193 - y1_output_root_mean_squared_error: 2.4737 - y2_output_root_mean_squared_error: 1.6490 - val_loss: 10.2193 - val_y1_output_loss: 6.3533 - val_y2_output_loss: 3.8659 - val_y1_output_root_mean_squared_error: 2.5206 - val_y2_output_root_mean_squared_error: 1.9662
Epoch 422/500
62/62 [==============================] - 0s 1ms/step - loss: 8.8310 - y1_output_loss: 6.0873 - y2_output_loss: 2.7437 - y1_output_root_mean_squared_error: 2.4672 - y2_output_root_mean_squared_error: 

Epoch 441/500
62/62 [==============================] - 0s 1ms/step - loss: 8.4878 - y1_output_loss: 5.7259 - y2_output_loss: 2.7619 - y1_output_root_mean_squared_error: 2.3929 - y2_output_root_mean_squared_error: 1.6619 - val_loss: 9.7936 - val_y1_output_loss: 6.0409 - val_y2_output_loss: 3.7527 - val_y1_output_root_mean_squared_error: 2.4578 - val_y2_output_root_mean_squared_error: 1.9372
Epoch 442/500
62/62 [==============================] - 0s 1ms/step - loss: 8.5223 - y1_output_loss: 5.7314 - y2_output_loss: 2.7909 - y1_output_root_mean_squared_error: 2.3940 - y2_output_root_mean_squared_error: 1.6706 - val_loss: 9.9023 - val_y1_output_loss: 6.0731 - val_y2_output_loss: 3.8292 - val_y1_output_root_mean_squared_error: 2.4644 - val_y2_output_root_mean_squared_error: 1.9568
Epoch 443/500
62/62 [==============================] - 0s 1ms/step - loss: 8.4638 - y1_output_loss: 5.6860 - y2_output_loss: 2.7778 - y1_output_root_mean_squared_error: 2.3845 - y2_output_root_mean_squared_error: 1

Epoch 462/500
62/62 [==============================] - 0s 1ms/step - loss: 8.0947 - y1_output_loss: 5.3255 - y2_output_loss: 2.7692 - y1_output_root_mean_squared_error: 2.3077 - y2_output_root_mean_squared_error: 1.6641 - val_loss: 10.2627 - val_y1_output_loss: 5.7696 - val_y2_output_loss: 4.4931 - val_y1_output_root_mean_squared_error: 2.4020 - val_y2_output_root_mean_squared_error: 2.1197
Epoch 463/500
62/62 [==============================] - 0s 1ms/step - loss: 8.1337 - y1_output_loss: 5.3536 - y2_output_loss: 2.7801 - y1_output_root_mean_squared_error: 2.3138 - y2_output_root_mean_squared_error: 1.6674 - val_loss: 9.5062 - val_y1_output_loss: 5.8147 - val_y2_output_loss: 3.6916 - val_y1_output_root_mean_squared_error: 2.4114 - val_y2_output_root_mean_squared_error: 1.9213
Epoch 464/500
62/62 [==============================] - 0s 1ms/step - loss: 8.0545 - y1_output_loss: 5.2951 - y2_output_loss: 2.7595 - y1_output_root_mean_squared_error: 2.3011 - y2_output_root_mean_squared_error: 

Epoch 483/500
62/62 [==============================] - 0s 1ms/step - loss: 7.8178 - y1_output_loss: 5.0748 - y2_output_loss: 2.7429 - y1_output_root_mean_squared_error: 2.2527 - y2_output_root_mean_squared_error: 1.6562 - val_loss: 9.4119 - val_y1_output_loss: 5.6205 - val_y2_output_loss: 3.7914 - val_y1_output_root_mean_squared_error: 2.3708 - val_y2_output_root_mean_squared_error: 1.9472
Epoch 484/500
62/62 [==============================] - 0s 1ms/step - loss: 7.8001 - y1_output_loss: 5.0510 - y2_output_loss: 2.7491 - y1_output_root_mean_squared_error: 2.2474 - y2_output_root_mean_squared_error: 1.6580 - val_loss: 10.1929 - val_y1_output_loss: 5.8691 - val_y2_output_loss: 4.3239 - val_y1_output_root_mean_squared_error: 2.4226 - val_y2_output_root_mean_squared_error: 2.0794
Epoch 485/500
62/62 [==============================] - 0s 1ms/step - loss: 7.7904 - y1_output_loss: 5.0595 - y2_output_loss: 2.7308 - y1_output_root_mean_squared_error: 2.2493 - y2_output_root_mean_squared_error: 